# imports

In [1]:
import glob
import pickle
import pandas as pd
from sklearn.cross_decomposition import CCA
import numpy as np


# First, I load the data…


In [2]:

file_paths = glob.glob("three_feature_folder/*.p")
#print(file_paths)
contents = []

for f in file_paths:
    contents.append(pickle.load(open(f,'rb')))
print(contents)
print(contents[0]['dm'].columns)
print(contents[0]['efel'].columns)
print(contents[0]['allen'].columns)




[{'efel':   AHP1_depth_from_peak AHP2_depth_from_peak  \
0  [88.38979998984621]  [87.28440000541352]   
0   [96.6853714067687]  [72.91153334421293]   

                                           AHP_depth  \
0  [-0.9102661308858728, 6.280333868327688, 7.221...   
0  [2.4339958693931294, 3.3257958690923175, 4.998...   

                                       AHP_depth_abs  \
0  [-73.61579999921395, -66.42520000000039, -65.4...   
0  [-70.27079999892545, -69.37899999922627, -67.7...   

                                  AHP_depth_abs_slow  \
0  [-31.46079997228933, -56.23471428545516, -57.1...   
0  [4.84379713387319, 6.76669334891591, 9.4894733...   

                                      AHP_depth_diff  \
0  [7.1905999992135605, 0.9412000001742911, 0.599...   
0  [0.8917999996991881, 1.6726000001982158, 1.466...   

                                 AHP_depth_from_peak  \
0  [88.38979998984621, 87.28440000541352, 86.6367...   
0  [96.6853714067687, 72.91153334421293, 72.55019...   

   


### Then, I do a cleanup step…
### fix fact that column names may be NU classes as opposed to strings.


In [3]:
print(contents[0]['model_id'])
print([c['model_id'] for c in contents if 'model_id' in c.keys()])

NMLCL001142
['NMLCL001142', 'NMLCL000973', 'NMLCL001474', 'NMLCL001457', 'NMLCL000341', 'NMLCL001496', 'NMLCL000777', 'NMLCL001525', 'NMLCL001500', 'NMLCL000133', 'NMLCL001069', 'NMLCL000541', 'NMLCL001515', 'NMLCL001438', 'NMLCL000501', 'NMLCL000373', 'NMLCL000309', 'NMLCL000637', 'NMLCL000405', 'NMLCL001539', 'NMLCL001519', 'NMLCL001507', 'NMLCL000937', 'NMLCL001503', 'NMLCL001464', 'NMLCL001138', 'NMLCL001469', 'NMLCL000229', 'NMLCL001139', 'NMLCL000101', 'NMLCL001442', 'NMLCL001101', 'NMLCL001037', 'NMLCL000701', 'NMLCL001450', 'NMLCL000905', 'NMLCL001510', 'NMLCL000509', 'NMLCL001141', 'NMLCL000437', 'NMLCL001434', 'NMLCL001146', 'NMLCL000809', 'NMLCL001145', 'NMLCL001140', 'NMLCL001136', 'NMLCL001488', 'NMLCL001482', 'NMLCL001542', 'NMLCL001147', 'NMLCL000873', 'NMLCL000261', 'NMLCL001445', 'NMLCL000165', 'NMLCL001430', 'NMLCL001461']


# Now I try to align and merge the data frames

This is much like merging and aligning the excell spreadsheets.

I try to find features that should be the same across feature space like Coefficient of variation.

In [4]:
standard = contents[0]['efel'].iloc[0]
standard

AHP1_depth_from_peak                                            [88.38979998984621]
AHP2_depth_from_peak                                            [87.28440000541352]
AHP_depth                         [-0.9102661308858728, 6.280333868327688, 7.221...
AHP_depth_abs                     [-73.61579999921395, -66.42520000000039, -65.4...
AHP_depth_abs_slow                [-31.46079997228933, -56.23471428545516, -57.1...
AHP_depth_diff                    [7.1905999992135605, 0.9412000001742911, 0.599...
AHP_depth_from_peak               [88.38979998984621, 87.28440000541352, 86.6367...
AHP_slow_time                     [0.9622641509434068, 0.8793103448275862, 0.772...
AHP_time_from_peak                [0.8000000000001819, 1.0000000000002274, 1.099...
AP1_amp                                                          [82.5170352807665]
AP1_begin_voltage                                              [-67.74303529013424]
AP1_begin_width                                                 [2.600000000

In [5]:
strong = contents[0]['efel'].iloc[1]
strong

AHP1_depth_from_peak                                             [96.6853714067687]
AHP2_depth_from_peak                                            [72.91153334421293]
AHP_depth                         [2.4339958693931294, 3.3257958690923175, 4.998...
AHP_depth_abs                     [-70.27079999892545, -69.37899999922627, -67.7...
AHP_depth_abs_slow                [4.84379713387319, 6.76669334891591, 9.4894733...
AHP_depth_diff                    [0.8917999996991881, 1.6726000001982158, 1.466...
AHP_depth_from_peak               [96.6853714067687, 72.91153334421293, 72.55019...
AHP_slow_time                     [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...
AHP_time_from_peak                [0.7000000000001592, 0.8000000000001819, 0.800...
AP1_amp                                                         [89.14303806413449]
AP1_begin_voltage                                              [-62.72846665629125]
AP1_begin_width                                                [1.1000000000

In [6]:
#.iloc[0]
#contents[0]['allen'][0]['width']#.iloc[1]
#contents[0]['allen']['protocol']
contents[-4]['allen']

,0,protocol
adp_index,NaN,3.0
adp_t,NaN,3.0
adp_v,NaN,3.0
clipped,0.000000,3.0
downstroke,-117.533852,3.0
downstroke_index,100553.500000,3.0
downstroke_t,1.005535,3.0
downstroke_v,-26.920645,3.0
fast_trough_index,100628.000000,3.0
fast_trough_t,1.006280,3.0


In [10]:

full_allens = [c['allen'] for c in contents if 'median_isi' in c['allen'].transpose()]
ALLEN_CV15 = [ fa[0]['isi_cv'] for fa in full_allens ]

contents[-1]['allen'].keys()#['isi_cv']
#[0]['isi_cv']#.iloc[0]
#ALLEN_CV30 = contents[0]['allen']['isi_cv'][1]#.iloc[1]
#full_allens

for c in contents: 
    cont_ = c['allen'].transpose()
    #try:
    #    print('yes')
        #print(cont_['mean_isi'])
    #except:
    #    pass
    #if 'mea_isi' in cont_:
    #    print(c)
        
        

#cont_['mean_isi']
    #if 'median_isi' in c['allen']: 
#ALLEN_CV15
#full_allens[0][0].keys()#['median_isi']
type(ALLEN_CV15[0])

pandas.core.series.Series

In [8]:
CV_DM = contents[0]['dm']['ISICVTest'].iloc[0]
accom_DM = contents[0]['dm']['InitialAccommodationMeanTest'].iloc[0]
inrh = contents[0]['dm']['InputResistanceTest'].iloc[0]
delay = contents[0]['dm']['AP1DelayMeanTest'].iloc[0]

# Allen


ahp = contents[0]['efel']['AHP_depth']
spwidth = contents[0]['efel']['spike_half_width']


adaptation_index_EFEL = contents[0]['efel']['adaptation_index']
daptation_index2_EFEL = contents[0]['efel']['adaptation_index2']
sag_amplitude_EFEL = contents[0]['efel']['sag_amplitude']
irregularity_index_EFEL = contents[0]['efel']['irregularity_index']
time_to_first_spike_EFEL = contents[0]['efel']['time_to_first_spike']

EFEL_ISI_CV_15 = contents[0]['efel']['ISI_CV'].iloc[0]
EFEL_ISI_CV_30 = contents[0]['efel']['ISI_CV'].iloc[1]

#aligned_f = pd.DataFrame(columns=['protocol','EFEL','DM','ALLEN'])
aligned_frame = []
#aligned_f = pd.DataFrame()
temp = {'protocol':1.5,'EFEL':float(EFEL_ISI_CV_15[0]),'DM':CV_DM,'ALLEN':ALLEN_CV15[0]}
#aligned_f.append(temp,ignore_index=True)#'cv1.5')

aligned_frame.append(temp)

temp = {'protocol':3.0,'ALLEN':ALLEN_CV30.iloc[-1],'DM':CV_DM,'EFEL':float(EFEL_ISI_CV_30[0])}
aligned_frame.append(temp)

#aligned_f.append(temp,ignore_index=True)#index='cv3.0')
#print(aligned_f)

#import pdb
#pdb.set_trace()
temp = {'protocol':3.0,'EFEL':adaptation_index_EFEL.iloc[-1],'DM':accom_DM,'ALLEN':contents[0]['allen']['adapt'].iloc[-1]}
#aligned_f = aligned_frame.append(temp)

temp = {'protocol':1.5,'EFEL':adaptation_index_EFEL.iloc[0],'DM':accom_DM,'ALLEN':contents[0]['allen']['adapt'].iloc[0]}
aligned_frame.append(temp)


temp = {'protocol':3.0,'EFEL':time_to_first_spike_EFEL.iloc[-1],'DM':delay,'ALLEN':contents[0]['allen']['latency'].iloc[-1]}
aligned_frame.append(temp)
temp = {'protocol':1.5,'EFEL':time_to_first_spike_EFEL.iloc[0],'DM':delay,'ALLEN':contents[0]['allen']['latency'].iloc[0]}
aligned_frame.append(temp)

af = pd.DataFrame(aligned_frame)
af.rename(index={1:'CV3'}, inplace=True)
af.rename(index={0:'CV15'}, inplace=True)
af.rename(index={3:'adaption15'}, inplace=True)
af.rename(index={2:'adaption3'}, inplace=True)
af.rename(index={2:'delay3'}, inplace=True)
af.rename(index={2:'delay15'}, inplace=True)

#print(af['ALLEN'].corr(af['DM']))

#print(af.corr())
print(af)


NameError: name 'ALLEN_CV30' is not defined

In [ ]:
import matplotlib.pyplot as plt

def plot_three_vectors(three_vectors,title=None):
    plot_vector = [i for i,_ in enumerate(three_vectors[0])]
    for tv in three_vectors:
        tv = (tv - np.mean(tv))/np.std(tv)
        plt.plot(plot_vector,tv)
        plt.title(title)
    plt.show()
    return

efel_CV = [ float(cont['efel']['ISI_CV'].iloc[0]) for cont in contents ]
dm_CV = [ float(cont['dm']['ISICVTest'].iloc[0]) for cont in contents ]
allen_CV = [ float(cont['allen']['isi_cv'].iloc[0]) for cont in contents ]

scaled_allen_CV = (allen_CV - np.mean(allen_CV))/np.std(allen_CV)
scaled_dm_CV = (dm_CV - np.mean(dm_CV))/np.std(dm_CV)
#sklearn.preprocessing.scale(, axis=0, with_mean=True, with_std=True, copy=True)
scaled_efel_CV = (efel_CV - np.mean(efel_CV))/np.std(efel_CV)
three_vectors = [scaled_allen_CV, scaled_dm_CV,scaled_efel_CV]
plot_three_vectors(three_vectors,title='CV')

In [ ]:



#sklearn.preprocessing.scale(efel_CV, axis=0, with_mean=True, with_std=True, copy=True)

plt.show()
X = list(zip(efel_CV, dm_CV))
Y = list(zip(dm_CV,allen_CV))
cca.fit(X,Y)

cca.coef_.shape                   # (5,5)

U_c, V_c = cca.transform(X, Y)

U_c.shape                         # (100,1)
V_c.shape                         # (100,1)
##
# wrong because needs different trace
##
inrh = contents[0]['efel']['ohmic_input_resistance']
##
# DM
CV_DM = contents[0]['dm']['ISICVTest'].iloc[0]
accom_DM = contents[0]['dm']['InitialAccommodationMeanTest'].iloc[0]
inrh = contents[0]['dm']['InputResistanceTest'].iloc[0]
delay = contents[0]['dm']['AP1DelayMeanTest'].iloc[0]

# Allen
ALLEN_CV15 = contents[0]['allen']['isi_cv'][0]#.iloc[0]
ALLEN_CV30 = contents[0]['allen']['isi_cv'][1]#.iloc[1]


'''

cca = CCA(n_components=1)
cca.fit(U, V)

cca.coef_.shape                   # (5,5)

U_c, V_c = cca.transform(U, V)
U_c.shape                         # (100,1)
V_c.shape                         # (100,1)
'''

In [ ]:
ALLEN_CV30